# Export JRC Permanent water GeoTIFFs to Google Drive
Export JRC Permanent water GeoTIFFs to Google Drive for each event (storm+region).

Files are saved to Google Service Account Drive, to access them use PyDrive to access the Google Drive API (requires enabling).
 * [Instructions to set up](https://stackoverflow.com/questions/71834208/why-is-the-export-to-the-drive-empty-when-using-google-earth-engine-in-python-w)
 * [Instructions for constructing queries](https://developers.google.com/drive/api/guides/search-files)

# Get JRC permanent water for the AoI

In [1]:
from os.path import join
import pandas as pd
import geopandas as gpd


wd = join('..', 'data')

current_datasets = pd.read_csv(join(wd, 'csvs', 'current_datasets.csv'))
ems_datasets = current_datasets[current_datasets.source == 'Copernicus EMS']
ems_events = [(row.event, row.region) for _, row in ems_datasets.iterrows()]

In [ ]:
import ee
import collections
collections.Callable = collections.abc.Callable

import time

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials

# initialize GEE
try:
    ee.Initialize()
except:
    service_account = "hybrid-models@hybridmodels-354115.iam.gserviceaccount.com"
    credentials = ee.ServiceAccountCredentials(service_account, join("gcloud_keys", ".hybridmodels-354115-e71f122c7f06.json"))
    ee.Initialize(credentials)

In [79]:
def get_gee_jrc(band, aoi_ee, event_str):
    jrc_permwater = (ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
                     .clip(aoi_ee)
                     .select(band)
                     .unmask(0))

    assexport_drive = ee.batch.Export.image.toDrive(jrc_permwater,
                                                    description=f'JRC_{event_str}',
                                                    fileNamePrefix=f"JRC{band}_{event_str}",
                                                    scale=10,
                                                    maxPixels=1e13)

    # export to google earth engine account
    assexport_drive.start()

    while (assexport_drive.status()['state'] == 'RUNNING') or (assexport_drive.status()['state'] == 'READY'):
        print('Still running...')
        time.sleep(30)

    print(f"Finished! Status: {assexport_drive.status()['state']}")
    
    
    
def download_from_drive():
    # authenticate to Google Drive (of the Service account)
    gauth = GoogleAuth()
    scopes = ['https://www.googleapis.com/auth/drive']
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name(join("gcloud_keys", ".hybridmodels-354115-e71f122c7f06.json"), scopes=scopes)

    drive = GoogleDrive(gauth)

    # get list of files
    # construct query: 
    file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
    for file in file_list:

        filename = file['title']

        # download file into working directory (in this case a tiff-file)
        file.GetContentFile(filename, mimetype="image/tiff")

        # delete file afterwards to keep the Drive empty
        file.Delete()

In [128]:
# use this line to just do a specific storm
ems_events = [('batsirai', 'menabe')]



for i in range(len(ems_events)):
    storm = ems_events[i][0]
    region = ems_events[i][1]
    event_str = '_'.join([storm, region])

    flood = gpd.read_file(join(wd, event_str, 'flood.gpkg'))
    aoi = gpd.read_file(join(wd, event_str, 'areaOfInterest.gpkg'))['geometry'][0]

    # convert aoi to a GEE Feature Collection
    aoi_ee = ee.Geometry.Polygon(aoi.__geo_interface__["coordinates"],
                                 proj=ee.Projection('EPSG:4326'))
    location = aoi_ee.centroid().coordinates().getInfo()[::-1]

    get_gee_jrc('occurrence', aoi_ee, event_str)
    ! say done sub loop

download_from_drive()


Still running...
Still running...
Still running...
Still running...
Finished! Status: COMPLETED


# Analysis (with help vectorising from QGIS)

In [ ]:
import matplotlib.pyplot as plt
from viz_utils import soge_colours

In [130]:
jrc_occurrence = gpd.read_file(join(wd, event_str, 'jrc_occurrence.gpkg'))

fig, ax = plt.subplots()
flood.plot(color=soge_colours['orange'], ax=ax, zorder=2)
jrc_occurrence.plot('occurrence', cmap='YlGnBu', ax=ax, zorder=1)

In [132]:
# get intersection and union of polygons
intersection = gpd.overlay(flood, jrc_occurrence, how='intersection')
union = gpd.overlay(flood, jrc_occurrence, how='union')

# get areas
intersection_area = intersection.to_crs(3857).area.sum()
flooded_area = flood.to_crs(3857).area.sum()
union_area = union.to_crs(3857).area.sum()

# percentage area
perc_area = intersection_area / flooded_area

# intersection over union
iou = intersection_area / union_area

# mean occurrence value in flood polygon
mean_occurence = intersection.occurrence.mean()
quantiles_occurrence = intersection.occurrence.quantile([0.25,0.5,0.75])

In [133]:
print(f"Results:\n-------\n")
print(f"%area: {perc_area:.2%}\nIoU: {iou:.2%}\nMean occurrence value in flood: {mean_occurence:.2f}")
for quantile, value in quantiles_occurrence.items():
    print(f"{quantile:.0%} quantile: {value:.2f}")

Results:
-------

%area: 13.71%
IoU: 0.15%
Mean occurrence value in flood: 6.52
25% quantile: 3.00
50% quantile: 6.00
75% quantile: 9.00
